In [23]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint

In [24]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [26]:
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [27]:
username2_category["kod8net"]

'tech'

In [28]:
'''train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]




# Load the additional CSV file
additional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"
additional_data_df = pd.read_csv(additional_data_path)

# Extract and rename the relevant columns
additional_data_df = additional_data_df[['Unnamed: 0', 'influencerCategory']].rename(columns={
    'Unnamed: 0': 'user_id', 
    'influencerCategory': 'category'
})

additional_data_df = additional_data_df.dropna()



# Convert the 'category' column to string type and apply .lower(), handling any NaN or unexpected values
additional_data_df['category'] = additional_data_df['category'].astype(str).fillna('').apply(str.lower)

# Append the new data to the original train_classification_df
train_classification_df = pd.concat([train_classification_df, additional_data_df], ignore_index=True)

# Update the username2_category dictionary with the new data
username2_category.update(additional_data_df.set_index("user_id").to_dict()["category"])

# Check the updated data by viewing the first few rows
print(train_classification_df.head())

# Re-check the category distribution
train_classification_df.groupby("category").count()'''


'train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)\ntrain_classification_df = train_classification_df.rename(columns={\'Unnamed: 0\': \'user_id\', \'label\': \'category\'})\n\n# Unifying labels\ntrain_classification_df["category"] = train_classification_df["category"].apply(str.lower)\nusername2_category = train_classification_df.set_index("user_id").to_dict()["category"]\n\n\n\n\n# Load the additional CSV file\nadditional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"\nadditional_data_df = pd.read_csv(additional_data_path)\n\n# Extract and rename the relevant columns\nadditional_data_df = additional_data_df[[\'Unnamed: 0\', \'influencerCategory\']].rename(columns={\n    \'Unnamed: 0\': \'user_id\', \n    \'influencerCategory\': \'category\'\n})\n\nadditional_data_df = additional_data_df.dropna()\n\n\n\n# Convert the \'category\' column to string type and apply .lower(), handling any NaN or unexpec

In [29]:
train_data_path = "/Users/Osama/Downloads/CS412PROJ/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile

In [30]:
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.shape

(2741, 44)

In [31]:
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji


def preprocess_text(text: str):
    

    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    #text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



#custom_stopwords = list(set(turkish_stopwords).union({
 #   'the', 'and', 'with', 'for', 'you', 'to', 'of', 'in', 'our', 'your', 'is', 'are','bir'
#}))
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=15000,min_df=10,sublinear_tf=True,ngram_range=(1, 3))

# fit the vectorizer
vectorizer.fit(corpus)

# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
feature_names = vectorizer.get_feature_names_out()

# Inspect the frequency of each word
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# Show the most frequent words (words in many posts)
print(df_tfidf.sum().sort_values(ascending=False).head(30))




test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

bir            81.180941
olsun          47.673401
yeni           43.671834
the            43.394632
olarak         40.511140
olan           40.352158
kutlu          38.237404
kutlu olsun    37.899475
güzel          37.530186
iyi            37.429708
kadar          36.025381
özel           35.035424
istanbul       34.763753
and            34.632501
of             34.231987
devam          34.096361
günü           33.739565
to             33.220320
birlikte       33.028780
in             32.498219
gün            32.468781
büyük          32.421884
var            32.042348
ilk            31.898787
ekim           31.048388
kasım          30.964692
yıl            30.016602
with           28.849120
bilgi          28.435614
teşekkür       28.348610
dtype: float64


In [33]:
# Making sure everything is fine
assert y_train.count("NA") == 0


In [34]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ability to', 'able to', 'abone olmayı', ..., 'şık tasarımı',
       'şık zarif', 'şıklığı bir'], dtype=object)

In [35]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,ability to,able to,abone olmayı,about our,about the,access to,accommodation experience,accompanied by,accompanied by the,according to,...,şükranlarımızı sunuyoruz,şükürler olsun,şık bir,şık konforlu,şık modern,şık rahat,şık tasarım,şık tasarımı,şık zarif,şıklığı bir
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df_tfidf.shape

(2741, 10604)

In [37]:
print("Shape of df_tfidf:", df_tfidf.shape)
print("Length of y_train:", len(y_train))

Shape of df_tfidf: (2741, 10604)
Length of y_train: 2741


In [57]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train,random_state=42)

In [39]:
x_train.shape

(2192, 10604)

In [40]:
x_val.shape

(549, 10604)

In [58]:
'''from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


model = MultinomialNB()
model.fit(x_train, y_train)'''
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import numpy as np
from collections import Counter

# Count the number of users in each class
class_counts = Counter(y_train)

# Total number of samples
total_samples = len(y_train)

# Calculate class weights using log10
'''log_class_weights = {
    cls: np.log10(1 + total_samples / count)  # Logarithmic scaling of class weight
    for cls, count in class_counts.items()
}
'''
unique_classes, class_counts = np.unique(y_train, return_counts=True)
log_class_weights = {cls: 1 / (np.log(cnt) / np.log(20)) for cls, cnt in zip(unique_classes, class_counts)}
print("Class Weights with Log10 scaling:", log_class_weights)

class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train), 
    y=y_train
)
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}
print("Class Weights:", class_weight_dict)


from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

# Initialize SVM model with class weights
svm_model = LinearSVC(class_weight=log_class_weights,max_iter=2000,random_state=123)



#svm_model = LinearSVC(class_weight=class_weight_dict, max_iter=2000)
#non_linear_svm = SVC(kernel='rbf', class_weight=log_class_weights,max_iter=2000,random_state=42)
# Train the model
svm_model.fit(x_train, y_train)

Class Weights with Log10 scaling: {np.str_('art'): np.float64(0.5955211694024377), np.str_('entertainment'): np.float64(0.5394838964231858), np.str_('fashion'): np.float64(0.5470194853204887), np.str_('food'): np.float64(0.4981500114024804), np.str_('gaming'): np.float64(1.301029995663981), np.str_('health and lifestyle'): np.float64(0.49958412562650956), np.str_('mom and children'): np.float64(0.6268371758695794), np.str_('sports'): np.float64(0.6657464410787219), np.str_('tech'): np.float64(0.532667665109185), np.str_('travel'): np.float64(0.5487105689237657)}
Class Weights: {np.str_('art'): np.float64(1.4326797385620915), np.str_('entertainment'): np.float64(0.8496124031007752), np.str_('fashion'): np.float64(0.9171548117154812), np.str_('food'): np.float64(0.5359413202933986), np.str_('gaming'): np.float64(21.92), np.str_('health and lifestyle'): np.float64(0.545273631840796), np.str_('mom and children'): np.float64(1.842016806722689), np.str_('sports'): np.float64(2.43555555555555

LinearSVC(class_weight={np.str_('art'): np.float64(0.5955211694024377),
                        np.str_('entertainment'): np.float64(0.5394838964231858),
                        np.str_('fashion'): np.float64(0.5470194853204887),
                        np.str_('food'): np.float64(0.4981500114024804),
                        np.str_('gaming'): np.float64(1.301029995663981),
                        np.str_('health and lifestyle'): np.float64(0.49958412562650956),
                        np.str_('mom and children'): np.float64(0.6268371758695794),
                        np.str_('sports'): np.float64(0.6657464410787219),
                        np.str_('tech'): np.float64(0.532667665109185),
                        np.str_('travel'): np.float64(0.5487105689237657)},
          max_iter=2000, random_state=123)

In [42]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Calculate class weights using log10
unique_classes, class_counts = np.unique(y_train, return_counts=True)
class_weights = {cls: 1 / np.log10(1 + cnt) for cls, cnt in zip(unique_classes, class_counts)}

print("Class Weights (log10 adjusted):", class_weights)

# Create Logistic Regression model
log_reg = LogisticRegression(
    multi_class='multinomial',  # for multiclass classification
    solver='lbfgs',            # efficient solver for multiclass problems
    class_weight=class_weights,  # handle class imbalance
    max_iter=1000               # allow sufficient iterations for convergence
)

# Train the model
log_reg.fit(x_train, y_train)

# Predict on validation set
y_val_pred = log_reg.predict(x_val)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_val, y_val_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
'''


'from sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import classification_report, confusion_matrix\nimport numpy as np\n\n# Calculate class weights using log10\nunique_classes, class_counts = np.unique(y_train, return_counts=True)\nclass_weights = {cls: 1 / np.log10(1 + cnt) for cls, cnt in zip(unique_classes, class_counts)}\n\nprint("Class Weights (log10 adjusted):", class_weights)\n\n# Create Logistic Regression model\nlog_reg = LogisticRegression(\n    multi_class=\'multinomial\',  # for multiclass classification\n    solver=\'lbfgs\',            # efficient solver for multiclass problems\n    class_weight=class_weights,  # handle class imbalance\n    max_iter=1000               # allow sufficient iterations for convergence\n)\n\n# Train the model\nlog_reg.fit(x_train, y_train)\n\n# Predict on validation set\ny_val_pred = log_reg.predict(x_val)\n\n# Evaluate the model\nprint("Classification Report:")\nprint(classification_report(y_val, y_val_pred))\n\n# Confus

In [43]:
'''from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(svm_model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)'''


"from sklearn.model_selection import GridSearchCV\nparam_grid = {'svc__C': [1, 5, 10, 50],\n              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}\ngrid = GridSearchCV(svm_model, param_grid)\n\n%time grid.fit(Xtrain, ytrain)\nprint(grid.best_params_)"

In [59]:
#@title Train Data
y_train_pred = svm_model.predict(x_train)

print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, zero_division=0))

Accuracy: 0.9949817518248175

Classification Report:
                      precision    recall  f1-score   support

                 art       1.00      0.99      1.00       153
       entertainment       1.00      1.00      1.00       258
             fashion       0.96      1.00      0.98       239
                food       1.00      1.00      1.00       409
              gaming       1.00      1.00      1.00        10
health and lifestyle       1.00      0.99      0.99       402
    mom and children       1.00      1.00      1.00       119
              sports       1.00      1.00      1.00        90
                tech       1.00      1.00      1.00       277
              travel       1.00      0.99      1.00       235

            accuracy                           0.99      2192
           macro avg       1.00      1.00      1.00      2192
        weighted avg       1.00      0.99      1.00      2192



In [60]:
y_val_pred = svm_model.predict(x_val)

print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

Accuracy: 0.6939890710382514

Classification Report:
                      precision    recall  f1-score   support

                 art       0.29      0.11      0.15        38
       entertainment       0.59      0.42      0.49        65
             fashion       0.57      0.75      0.65        60
                food       0.87      0.94      0.91       102
              gaming       0.00      0.00      0.00         3
health and lifestyle       0.67      0.77      0.72       100
    mom and children       0.68      0.43      0.53        30
              sports       0.82      0.78      0.80        23
                tech       0.72      0.83      0.77        69
              travel       0.68      0.75      0.71        59

            accuracy                           0.69       549
           macro avg       0.59      0.58      0.57       549
        weighted avg       0.67      0.69      0.67       549

Confusion Matrix:
 [[ 4  5 13  1  0  6  1  1  4  3]
 [ 4 27  5  5  0 11  0  

In [ ]:

from sklearn.feature_selection import chi2

chi2_scores, p_values = chi2(x_train, y_train)
feature_ranks = pd.DataFrame({
    "feature": feature_names,
    "chi2": chi2_scores
}).sort_values(by="chi2", ascending=False)

# Display top features
print(feature_ranks.head(50))

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_val, y_val_pred, labels=np.unique(y_train))
conf_matrix_df = pd.DataFrame(
    conf_matrix, index=np.unique(y_train), columns=np.unique(y_train)
)

print("Confusion Matrix:")
print(conf_matrix_df)

# Analyze specific misclassifications
print("Misclassified samples:")
for i, (true, pred) in enumerate(zip(y_val, y_val_pred)):
    if true != pred:
        print(f"Sample {i}: True={true}, Predicted={pred}")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create bigrams and trigrams
vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # This will generate unigrams, bigrams, and trigrams
X_train_tfidf = vectorizer.fit_transform(x_train)

# Apply chi-squared test again on the new features
chi2_scores, p_values = chi2(X_train_tfidf, y_train)

# Create a dataframe of the features and their chi2 scores
feature_names = vectorizer.get_feature_names_out()
feature_ranks = pd.DataFrame({
    "feature": feature_names,
    "chi2": chi2_scores
}).sort_values(by="chi2", ascending=False)

# Display top features
print(feature_ranks.head(20))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
cm = confusion_matrix(y_val, y_val_pred)

# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
# Ensure that validation data is properly referenced
# Assuming you have a list of usernames and captions for the validation set
val_usernames = train_usernames[len(x_train):]  # Example placeholder
val_corpus = corpus[len(x_train):]  # Example placeholder

# Ensure y_val and y_val_pred are aligned with validation usernames
y_val_aligned = y_val[:len(val_usernames)]
y_val_pred_aligned = y_val_pred[:len(val_usernames)]

# Recreate the DataFrame for validation data
val_data = pd.DataFrame({
    "Username": val_usernames,
    "True Label": y_val_aligned,
    "Predicted Label": y_val_pred_aligned,
    "Captions": val_corpus
})

# Identify misclassified samples
misclassified = val_data[val_data["True Label"] != val_data["Predicted Label"]]

# Print misclassified samples with their captions
print("Misclassified Samples with Captions:")
for _, row in misclassified.iterrows():
    print(f"Username: {row['Username']}")
    print(f"True Label: {row['True Label']}")
    print(f"Predicted Label: {row['Predicted Label']}")
    print(f"Captions: {row['Captions']}\n")


In [ ]:
#@title Test Data
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-classification-round1.dat"

with open(test_data_path, "rt") as fh:
    for i, line in enumerate(fh):
        print(line.strip())
        if i >= 4:  # Stop after 5 lines
            break

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

In [ ]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


test_unames.remove("screenname")

In [ ]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

In [ ]:
'''import gzip
import json

# Open the gzip-compressed JSONL file
with gzip.open('training-dataset.jsonl.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        # Each line is a JSON object, so load it
        data = json.loads(line)
        print(data)  # You can process the data as needed
'''